# Imports

In [1]:
print('Loading libraries')
start_time <- Sys.time()
suppressPackageStartupMessages({
    library(data.table)
    library(ggplot2)
    library(dplyr)
    library(stringr)
    library(rstatix)
    library(tidyr)
    library(phenoptr)
    }
)
print('Library load finished')
print(Sys.time()-start_time)


hpc =  TRUE
work_desktop = FALSE
work_linux = TRUE
singularity = TRUE
prj =  "HCC-CBS-173-Hillman-BFerris-NRG-HN003-Vectra"
if (!hpc){
    if (work_desktop){
        if (work_linux){
            base = paste0('/mnt/d/OneDrive - University of Pittsburgh/Internal_Project_Data_2/' ,prj)
        }else{
            base =paste0('D:\\OneDrive - University of Pittsburgh\\Internal_Project_Data_2\\', prj)
        }
    }else{
        base = paste0('/home/brian/data/odrive/Internal_Project_Data_2/',prj)
    }
}else{
    if (singularity){
        base = '/mnt'
    }else{
        base = paste0('/ix/rbao/Projects/', prj)
    }
}
print(base)
inform.data <- file.path(base,'data','inform_data_exports')
results <- file.path(base,'results')
print(results)
fn <- 'cell_density_34_all_inc_tumor_stroma_20_percent_v1.csv'
df <- read.csv(file.path(results,fn))
head(df)


[1] "Loading libraries"
[1] "Library load finished"
Time difference of 8.928175 secs
[1] "/mnt"
[1] "/mnt/results"


,Sample.Name,study_id,total_roi,n_tumor_segments,n_stroma_segments,PDL1.panCK.tumor_percent,PDL1.panCK.stroma_percent,PDL1.panCK.all_percent,PDL1.panCKneg.CD3pos.tumor_percent,PDL1.panCKneg.CD3pos.stroma_percent,PDL1.panCKneg.CD3pos.all_percent,PDL1.panCKneg.CD3neg.tumor_percent,PDL1.panCKneg.CD3neg.stroma_percent,PDL1.panCKneg.CD3neg.all_percent,CD3.CD8.Tcell.tumor_percent,CD3.CD8.Tcell.stroma_percent,CD3.CD8.Tcell.all_percent,Treg.tumor_percent,Treg.stroma_percent,Treg.all_percent
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,RACTIH_Scan1.qptiff,RACTIH,12,12,12,1.581028,2.4818933,1.932385,0.002949678,0.01845274,0.008996204,0.4277034,2.1220649,1.0885406,0.020647749,0.009226369,0.01619317,0.000000000,0.004613184,0.001799241
2,RACTKE_Scan1.qptiff,RACTKE,285,285,285,68.519584,35.4576429,51.416619,0.049980291,1.07511932,0.580285409,1.1840614,12.9683078,7.2800601,0.055449832,2.010679038,1.06689123,0.092793596,3.922979994,2.074151626
3,RACTLH_Scan2.qptiff,RACTLH,59,59,59,8.792360,3.9117319,6.129775,0.000000000,0.02801540,0.015283563,0.3746256,2.2418837,1.3932923,0.003128397,0.319896799,0.17593870,0.004692596,0.138773968,0.077839544
4,RACTNY_Scan1.qptiff,RACTNY,27,27,27,3.737032,0.8999784,2.049500,0.004234597,0.02019182,0.013726247,0.3917002,0.7427706,0.6005233,0.406521279,1.400447105,0.99772659,0.175735761,0.346145525,0.277098615
5,RACTRF_Scan1.qptiff,RACTRF,10,10,10,13.805149,1.4236082,8.444917,0.058222280,0.14405559,0.095381342,0.2393583,0.7880688,0.4769067,0.226419977,1.559189899,0.80340438,2.083063786,5.363952207,3.503430060
6,RACTRU_Scan1.qptiff,RACTRU,38,38,38,23.019207,8.3016812,11.614162,0.036987134,0.47420639,0.375801253,1.4504240,7.0893089,5.8201625,2.079205305,1.963582790,1.98960600,0.198145359,1.952840251,1.557910260


In [32]:
fns = list.files(inform.data,pattern = '*cell_seg_data.txt')
length(fns)
print(fns[100])

[1] 2029

[1] "RACTKE_Scan1_[12997,52179]_cell_seg_data.txt"


# Load example inForm cell_seg output .txt file:

Note: confirmed there are tumor and stroma regions in this file

In [2]:
csd <- read.csv(file.path(inform.data,'RACTIH_Scan1_[16434,44304]_cell_seg_data.txt'),
               sep = '\t',
               check.names=FALSE) #check.names=FALSE will prevent replacing spaces with periods
head(csd,n=2)

,Path,Sample Name,Tissue Category,Phenotype-CD3,Phenotype-PDL1,Phenotype-CK,Phenotype-CD8,Phenotype-PD1,Phenotype-FOXP3,Cell ID,⋯,TMA Row,TMA Column,TMA Field,Confidence-CD3,Confidence-PDL1,Confidence-CK,Confidence-CD8,Confidence-PD1,Confidence-FOXP3,inForm 2.6.7900.18857
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>
1,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Stroma,other,other,other,other,other,other,1,⋯,0,0,0,99.72%,100.00%,99.98%,99.80%,99.67%,100.00%,NA
2,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,PD1+,other,2,⋯,0,0,0,99.73%,99.97%,100.00%,96.40%,93.45%,99.97%,NA


# Reformat the cell_seg to contain new unified Phenotype column

In [3]:
unique(csd[,'Tissue Category'])

[1] "Tumor"  "Stroma" "Blank"

In [3]:
p <- c('Phenotype-CK','Phenotype-PDL1','Phenotype-CD3',
       'Phenotype-CD8','Phenotype-PD1','Phenotype-FOXP3')
csd$Phenotype <- ''
for (pheno in p){
    temp = csd[,pheno]
    temp[temp=='other'] = ''
    csd$Phenotype <- paste0(csd$Phenotype,temp)
    }
dim(csd)
# Remove cells with no labeling:

clean.csd <- csd[csd[,'Tissue Category'] != 'Blank',]
clean.csd <- clean.csd %>% filter(Phenotype != '')
dim(clean.csd)

# Remove CK+ cell labeling false alarms:
idx.1 <- clean.csd$Phenotype %like% 'CK+'
print(sum(idx.1))
idx.2 <- !(clean.csd$Phenotype %in% c('CK+','CK+PDL1+','CK+PD1+'))
idx <- idx.1 & idx.2
clean.csd[idx,'Phenotype'] <- substring(clean.csd[idx,'Phenotype'],
                                          first=4,last=nchar(clean.csd$Phenotype[idx]))

# Count
clean.csd %>% count(Phenotype)
# clean.csd %>% count('Tissue Category')

[1] 5418  218

[1] 2503  218

[1] 2308


Phenotype,n
<chr>,<int>
CD3+,4
CD3+CD8+PD1+,1
CD3+PD1+,3
CD8+,35
CD8+PD1+,8
CK+,1234
CK+PD1+,1047
CK+PDL1+,4
FOXP3+,3


In [4]:
distances <- find_nearest_distance(clean.csd)
csd.dist <- bind_cols(clean.csd, distances)
csd.dist %>% group_by(Phenotype) %>% 
  select(Phenotype, starts_with('Distance to')) %>% 
  summarize_all(~round(mean(.), 1))

Phenotype,Distance to CD3+,Distance to CD3+CD8+PD1+,Distance to CD3+PD1+,Distance to CD8+,Distance to CD8+PD1+,Distance to CK+,Distance to CK+PD1+,Distance to CK+PDL1+,Distance to FOXP3+,Distance to PD1+,Distance to PD1+FOXP3+,Distance to PDL1+,Distance to PDL1+CD8+PD1+,Distance to PDL1+PD1+
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CD3+,166.0,313.6,308.2,72.4,143.0,18.0,26.8,255.1,239.6,34.9,120.2,60.9,325.6,282.6
CD3+CD8+PD1+,170.4,NA,209.2,10.0,97.5,10.4,15.5,177.5,128.6,32.9,87.6,76.2,220.9,192.8
CD3+PD1+,141.8,261.5,85.2,17.3,97.8,15.6,19.4,172.3,241.0,37.5,56.9,60.2,87.3,124.6
CD8+,193.9,295.5,325.4,28.4,143.7,21.2,29.6,305.3,192.7,48.8,103.5,72.5,379.4,231.5
CD8+PD1+,206.6,270.9,258.9,61.8,129.5,10.4,29.6,228.1,226.2,39.4,72.3,45.1,281.0,185.4
CK+,241.9,408.6,417.0,134.7,211.4,11.5,17.7,393.3,260.7,67.5,107.0,122.4,488.3,287.3
CK+PD1+,239.2,375.4,367.9,137.2,207.7,13.8,9.2,399.4,220.1,68.8,101.4,133.3,446.3,265.6
CK+PDL1+,246.0,239.5,220.2,83.6,81.0,6.6,19.8,139.6,300.5,31.3,61.3,18.9,231.1,179.7
FOXP3+,196.1,249.7,262.1,96.2,179.6,16.1,26.6,359.3,235.8,38.7,78.0,162.7,341.0,273.4


# Examine count_within() command

In [43]:
help(count_within)

count_within {phenoptr},R Documentation
csd,"A data frame with Cell X Position, Cell Y Position and Phenotype columns, such as the result of calling read_cell_seg_data."
"from, to",Selection criteria for the rows and columns. Accepts all formats accepted by select_rows.
radius,The radius or radii to search within.
category,Optional tissue category to restrict both from and to.
dst,"Optional distance matrix corresponding to csd, produced by calling distance_matrix."


# Distance matrix approach

In [14]:
dst <- distance_matrix(clean.csd) # Compute this just once and re-use it
dim(dst)

[1] 2503 2503

In [22]:
idx.dst <- (dst < 35)
sqrt(sum(idx.dst))


[1] 219.9295

In [25]:
?select_rows

select_rows {phenoptr},R Documentation
csd,A data frame
sel,"May be a character vector, a one-sided formula, a list containing such or NA. A character vector is interpreted as the name(s) of one or more phenotypes and selects any matching phenotype. A formula is interpreted as an expression on the columns of csd. Multiple list items are joined with AND. NA is interpreted as ""select all"". It is convenient for lists of selection criteria."


In [38]:
colnames(clean.csd)

[1] "Path"                                                                     
  [2] "Sample Name"                                                              
  [3] "Tissue Category"                                                          
  [4] "Phenotype-CD3"                                                            
  [5] "Phenotype-PDL1"                                                           
  [6] "Phenotype-CK"                                                             
  [7] "Phenotype-CD8"                                                            
  [8] "Phenotype-PD1"                                                            
  [9] "Phenotype-FOXP3"                                                          
 [10] "Cell ID"                                                                  
 [11] "Total Cells"                                                              
 [12] "Tissue Category Area (square microns)"                                    
 [13] "Cell Density (per square mm)"                                             
 [14] "Cell X Position"                                                          
 [15] "Cell Y Position"                                                          
 [16] "Process Region ID"                                                        
 [17] "Distance from Process Region Edge (microns)"                              
 [18] "Category Region ID"                                                       
 [19] "Distance from Tissue Category Edge (microns)"                             
 [20] "Nucleus Area (square microns)"                                            
 [21] "Nucleus Area (percent)"                                                   
 [22] "Nucleus Compactness"                                                      
 [23] "Nucleus Minor Axis"                                                       
 [24] "Nucleus Major Axis"                                                       
 [25] "Nucleus Axis Ratio"                                                       
 [26] "Nucleus DAPI (DAPI) Min (Normalized Counts, Total Weighting)"             
 [27] "Nucleus DAPI (DAPI) Mean (Normalized Counts, Total Weighting)"            
 [28] "Nucleus DAPI (DAPI) Max (Normalized Counts, Total Weighting)"             
 [29] "Nucleus DAPI (DAPI) Std Dev (Normalized Counts, Total Weighting)"         
 [30] "Nucleus DAPI (DAPI) Total (Normalized Counts, Total Weighting)"           
 [31] "Nucleus CD8 (Opal 480) Min (Normalized Counts, Total Weighting)"          
 [32] "Nucleus CD8 (Opal 480) Mean (Normalized Counts, Total Weighting)"         
 [33] "Nucleus CD8 (Opal 480) Max (Normalized Counts, Total Weighting)"          
 [34] "Nucleus CD8 (Opal 480) Std Dev (Normalized Counts, Total Weighting)"      
 [35] "Nucleus CD8 (Opal 480) Total (Normalized Counts, Total Weighting)"        
 [36] "Nucleus PD-L1 (Opal 520) Min (Normalized Counts, Total Weighting)"        
 [37] "Nucleus PD-L1 (Opal 520) Mean (Normalized Counts, Total Weighting)"       
 [38] "Nucleus PD-L1 (Opal 520) Max (Normalized Counts, Total Weighting)"        
 [39] "Nucleus PD-L1 (Opal 520) Std Dev (Normalized Counts, Total Weighting)"    
 [40] "Nucleus PD-L1 (Opal 520) Total (Normalized Counts, Total Weighting)"      
 [41] "Nucleus FOXP3 (Opal 570) Min (Normalized Counts, Total Weighting)"        
 [42] "Nucleus FOXP3 (Opal 570) Mean (Normalized Counts, Total Weighting)"       
 [43] "Nucleus FOXP3 (Opal 570) Max (Normalized Counts, Total Weighting)"        
 [44] "Nucleus FOXP3 (Opal 570) Std Dev (Normalized Counts, Total Weighting)"    
 [45] "Nucleus FOXP3 (Opal 570) Total (Normalized Counts, Total Weighting)"      
 [46] "Nucleus PD-1 (Opal 620) Min (Normalized Counts, Total Weighting)"         
 [47] "Nucleus PD-1 (Opal 620) Mean (Normalized Counts, Total Weighting)"        
 [48] "Nucleus PD-1 (Opal 620) Max (Normalized Counts, Total Weighting)"         
 [49] "Nucleus PD-1 (Opal 620) Std Dev (Normalized Counts, Total W

In [42]:
selector <- list('CK+', ~`Entire Cell PD-L1 (Opal 520) Total (Normalized Counts, Total Weighting)`>25)
pdl1.test <- select_rows(clean.csd, selector)
sum(pdl1.test)
clean.csd[pdl1.test,1:20]

[1] 1094

,Path,Sample Name,Tissue Category,Phenotype-CD3,Phenotype-PDL1,Phenotype-CK,Phenotype-CD8,Phenotype-PD1,Phenotype-FOXP3,Cell ID,Total Cells,Tissue Category Area (square microns),Cell Density (per square mm),Cell X Position,Cell Y Position,Process Region ID,Distance from Process Region Edge (microns),Category Region ID,Distance from Tissue Category Edge (microns),Nucleus Area (square microns)
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
5,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,7,NA,NA,NA,15987.0,43964.1,#N/A,#N/A,1,#N/A,61.6
7,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,10,NA,NA,NA,16076.8,43964.1,#N/A,#N/A,#N/A,#N/A,164.1
8,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Stroma,other,other,CK+,other,other,other,13,NA,NA,NA,16116.5,43963.6,#N/A,#N/A,#N/A,2.5,126.2
9,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,15,NA,NA,NA,16132.9,43962.1,#N/A,#N/A,2,#N/A,105.0
10,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,16,NA,NA,NA,16139.9,43974.1,#N/A,#N/A,#N/A,4.1,1017.4
11,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,17,NA,NA,NA,16162.7,43960.2,#N/A,#N/A,2,#N/A,30.3
13,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,19,NA,NA,NA,16200.9,43965.1,#N/A,#N/A,#N/A,2.0,248.6
15,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,22,NA,NA,NA,16262.0,43961.6,#N/A,#N/A,3,#N/A,62.8
16,D:\Data\VectraPolaris\NRG-HN003\RACTIH\Scan1,RACTIH_Scan1.qptiff,Tumor,other,other,CK+,other,other,other,26,NA,NA,NA,16309.6,43960.2,#N/A,#N/A,3,#N/A,47.3


In [44]:
# Try count_within() measure CD3+FOXP3+ to CK+

In [12]:
# Find the number of Tregs with a tumor cell 35 microns
for (pheno in unique(clean.csd$Phenotype)){
    print(pheno)
    outer_margin <- count_within(clean.csd, 
                                 from=pheno, 
                                 to='CK+', 
                                 radius=c(35),
                                 category = c('Stroma')
                                 )
    idx = outer_margin$from_with > 0
    print(outer_margin[idx,])
}
# outer_margin

[1] "CK+PD1+"
# A tibble: 1 × 5
  radius from_count to_count from_with within_mean
   <dbl>      <int>    <int>     <int>       <dbl>
1     35         13      160        11        1.54
[1] "CK+"
# A tibble: 1 × 5
  radius from_count to_count from_with within_mean
   <dbl>      <int>    <int>     <int>       <dbl>
1     35        160      160       124        1.62
[1] "PD1+"
# A tibble: 1 × 5
  radius from_count to_count from_with within_mean
   <dbl>      <int>    <int>     <int>       <dbl>
1     35         57      160        42        1.18
[1] "PDL1+"
# A tibble: 1 × 5
  radius from_count to_count from_with within_mean
   <dbl>      <int>    <int>     <int>       <dbl>
1     35         35      160        26        1.43
[1] "CK+PDL1+"
# A tibble: 1 × 5
  radius from_count to_count from_with within_mean
   <dbl>      <int>    <int>     <int>       <dbl>
1     35          1      160         1           2
[1] "CD8+"
# A tibble: 1 × 5
  radius from_count to_count from_with within_mean
   

In [5]:
pheno <- 'PD1+'
outer_margin <- count_within(clean.csd, from=pheno, to='CK+', 
                         radius=c(35,50,100,200),  category = c('Tumor'))
idx = outer_margin$from_with > 0
print(outer_margin[idx,])

# A tibble: 4 × 5
  radius from_count to_count from_with within_mean
   <dbl>      <int>    <int>     <int>       <dbl>
1     35         38     1074        38        6.71
2     50         38     1074        38       12.4 
3    100         38     1074        38       38.2 
4    200         38     1074        38      134.  


Workflow idea: is it possible to subset any cell with < 30um distance to tumor?

ultimately how do I determine N cells within 35um of tumor outer margin / inner margin? create version where tissue category is set as the phenotype? trying this:

In [80]:
temp <- clean.csd
temp$Phenotype <- temp[,'Tissue Category']

distances <- find_nearest_distance(temp)
temp <- bind_cols(temp, distances)
temp %>% group_by(Phenotype) %>% 
  select(Phenotype, starts_with('Distance to')) %>% 
  summarize_all(~round(mean(.), 1))
dist = 100
outer_margin <- (temp[,'Distance to Tumor'] < dist) && (temp[,'Tissue Category'] =='Stroma')
inner_margin <- (temp[,'Distance to Stroma'] < dist ) && (temp[,'Tissue Category'] =='Tumor')
print(sum(outer_margin))
print(sum(inner_margin))
# outer_margin <- count_within(temp, from='Stroma', to='Tumor', 
#                              radius=c(35))
# outer_margin

Phenotype,Distance to Blank,Distance to Stroma,Distance to Tumor
<chr>,<dbl>,<dbl>,<dbl>
Blank,NA,14.4,56.2
Stroma,347.9,12.1,24.2
Tumor,351.1,34.8,8.7


[1] 0
[1] 1


In [79]:
sum(temp[,'Tissue Category'] =='Stroma')

[1] 585

In [78]:
sum(temp[,'Distance to Tumor'] < 35 )

[1] 2791

# Loop through files and measure how many cells in the margins

In [97]:

dim(df)
head(df)

[1] 2029    2

,inner_n,outer_n
,<lgl>,<lgl>
"RACTIH_Scan1_[15509,42225]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[15509,42918]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[15509,43611]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[15509,44304]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[16434,42225]_cell_seg_data.txt",NA,NA
"RACTIH_Scan1_[16434,42918]_cell_seg_data.txt",NA,NA


In [99]:
p <- c('Phenotype-CK','Phenotype-PDL1','Phenotype-CD3',
   'Phenotype-CD8','Phenotype-PD1','Phenotype-FOXP3')
df <- data.frame(matrix(NA, nrow = length(fns), ncol = 2),row.names=fns)
colnames(df) <- c('inner_n','outer_n')
for (fn in fns){
    csd <- read.csv(file.path(inform.data,fn),
               sep = '\t',
               check.names=FALSE) #check.names=FALSE will prevent replacing spaces with periods\
    csd$Phenotype <- ''
    for (pheno in p){
        temp = csd[,pheno]
        temp[temp=='other'] = ''
        csd$Phenotype <- paste0(csd$Phenotype,temp)
        }
    # dim(csd)
    # Remove cells with no labeling:

    clean.csd <- csd[clean.csd[,'Tissue Category'] != 'Blank',]
    clean.csd <- clean.csd %>% filter(Phenotype != '')
    # dim(clean.csd)

    # Remove CK+ cell labeling false alarms (CK+CD3+CD8+ etc):
    idx.1 <- clean.csd$Phenotype %like% 'CK+'
    # print(sum(idx.1))
    idx.2 <- !(clean.csd$Phenotype %in% c('CK+','CK+PDL1+','CK+PD1+'))
    idx <- idx.1 & idx.2
    clean.csd[idx,'Phenotype'] <- substring(clean.csd[idx,'Phenotype'],
                                              first=4,last=nchar(clean.csd$Phenotype[idx]))
    
    temp <- clean.csd
    temp$Phenotype <- temp[,'Tissue Category']

    distances <- find_nearest_distance(temp)
    temp <- bind_cols(temp, distances)
    temp %>% group_by(Phenotype) %>% 
      select(Phenotype, starts_with('Distance to')) %>% 
      summarize_all(~round(mean(.), 1))
    dist = 100
    outer_margin <- (temp[,'Distance to Tumor'] < dist) && (temp[,'Tissue Category'] =='Stroma')
    inner_margin <- (temp[,'Distance to Stroma'] < dist ) && (temp[,'Tissue Category'] =='Tumor')
    df[fn,'inner_n'] <- sum(inner_margin)
    df[fn,'outer_n'] <- sum(outer_margin)
    }

ERROR: Error in `[.data.frame`(temp, , "Distance to Stroma"): undefined columns selected


In [ ]:
mean(df)

In [ ]:
help(mean)